In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-cutting

*Tinantyang paggamit: Walong minuto sa isang Eagle processor (PAUNAWA: Ito ay tantiya lamang. Maaaring mag-iba ang inyong runtime.)*

## Pinagmulan

Ipinakikita ng tutorial na ito kung paano bumuo ng `Qiskit pattern` para sa pagputol ng mga gate sa quantum circuit upang bawasan ang lalim ng circuit. Para sa mas malalim na talakayan tungkol sa pagputol ng circuit, bisitahin ang [circuit cutting Qiskit addon docs](https://qiskit.github.io/qiskit-addon-cutting/).

## Mga Kinakailangan

Bago magsimula sa tutorial na ito, siguraduhing mayroon kayong naka-install na sumusunod:
- Qiskit SDK v2.0 o mas bago, na may suportang [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 o mas bago (`pip install qiskit-ibm-runtime`)
- Circuit cutting Qiskit addon v0.9.0 o mas bago (`pip install qiskit-addon-cutting`)

## Pag-setup

In [1]:
import numpy as np

from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList, Statevector, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_addon_cutting import (
    cut_gates,
    generate_cutting_experiments,
    reconstruct_expectation_values,
)

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

## Hakbang 1: Magtakda ng classical inputs sa quantum problem
Isasagawa natin ang ating Qiskit pattern gamit ang apat na hakbang na nakabalangkas sa [docs](/guides/intro-to-patterns). Sa kasong ito, sisimulahin natin ang mga expectation value sa isang circuit na may tiyak na lalim sa pamamagitan ng pagputol ng mga gate na nagreresulta sa swap gates at pagsasagawa ng mga subexperiment sa mas mababaw na mga circuit. Ang pagputol ng gate ay nauugnay sa Hakbang 2 (mag-optimize ng circuit para sa quantum execution sa pamamagitan ng pagbubukod ng mga malayong gate) at 4 (post-processing upang muling itayo ang mga expectation value sa orihinal na circuit).
Sa unang hakbang, bubuo tayo ng circuit mula sa Qiskit circuit library at magtakda ng ilang mga observable.

*   Input: Mga classical parameter upang tukuyin ang isang circuit
*   Output: Abstract circuit at mga observable

In [2]:
circuit = EfficientSU2(num_qubits=4, entanglement="circular").decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])
circuit.draw("mpl", scale=0.8, style="iqp")

<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/54ed0f13-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/54ed0f13-0.avif)

## Hakbang 2: I-optimize ang problema para sa quantum hardware execution
*   Input: Abstract circuit at mga observable
*   Output: Target circuit at mga observable na ginawa sa pamamagitan ng pagputol ng mga malayong gate upang bawasan ang transpiled circuit depth

Pumipili tayo ng paunang layout na nangangailangan ng dalawang swap upang maisagawa ang mga gate sa pagitan ng mga qubit 3 at 0 at isa pang dalawang swap upang ibalik ang mga qubit sa kanilang mga paunang posisyon. Pumipili tayo ng `optimization_level=3`, na siyang pinakamataas na antas ng optimization na magagamit sa preset pass manager.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, min_num_qubits=circuit.num_qubits, simulator=False
)

pm = generate_preset_pass_manager(
    optimization_level=3, initial_layout=[0, 1, 2, 3], backend=backend
)
transpiled_qc = pm.run(circuit)

![Coupling map showing the qubits that will need to be swapped](../docs/images/tutorials/depth-reduction-with-circuit-cutting/swaps.avif)

In [4]:
print(f"Transpiled circuit depth: {transpiled_qc.depth()}")
transpiled_qc.draw("mpl", scale=0.4, idle_wires=False, style="iqp", fold=-1)

Transpiled circuit depth: 103


<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/4fe4af43-1.avif" alt="Output of the previous code cell" />

*Find and cut the distant gates:* We will replace the distant gates (gates connecting non-local qubits, 0 and 3) with `TwoQubitQPDGate` objects by specifying their indices. `cut_gates` will replace the gates in the specified indices with `TwoQubitQPDGate` objects and also return a list of `QPDBasis` instances -- one for each gate decomposition. The `QPDBasis` object contains information about how to decompose the cut gates into single-qubit operations.

In [5]:
# Find the indices of the distant gates
cut_indices = [
    i
    for i, instruction in enumerate(circuit.data)
    if {circuit.find_bit(q)[0] for q in instruction.qubits} == {0, 3}
]

# Decompose distant CNOTs into TwoQubitQPDGate instances
qpd_circuit, bases = cut_gates(circuit, cut_indices)

qpd_circuit.draw("mpl", scale=0.8)

<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/23e3d25e-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/4fe4af43-1.avif)

*Hanapin at putulin ang mga malayong gate:* Papalitan natin ang mga malayong gate (mga gate na nag-uugnay ng mga non-local qubit, 0 at 3) ng mga `TwoQubitQPDGate` object sa pamamagitan ng pagtukoy ng kanilang mga indise. Ang `cut_gates` ay papalitan ang mga gate sa tinukoy na mga indise ng mga `TwoQubitQPDGate` object at magbabalik din ng listahan ng mga `QPDBasis` instance -- isa para sa bawat gate decomposition. Ang `QPDBasis` object ay naglalaman ng impormasyon tungkol sa kung paano bubuuin ang mga pinutol na gate sa mga single-qubit operation.

In [6]:
# Generate the subexperiments and sampling coefficients
subexperiments, coefficients = generate_cutting_experiments(
    circuits=qpd_circuit, observables=observables, num_samples=np.inf
)

![Output of the previous code cell](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/23e3d25e-0.avif)

*Bumuo ng mga subexperiment na isasagawa sa backend*: Tumatanggap ang `generate_cutting_experiments` ng circuit na naglalaman ng mga `TwoQubitQPDGate` instance at mga observable bilang `PauliList`.

Upang simulahin ang expectation value ng buong circuit, maraming subexperiment ang nabubuo mula sa joint quasiprobability distribution ng mga nabukod na gate at pagkatapos ay isinasagawa sa isa o higit pang mga backend. Ang bilang ng mga sample na kinuha mula sa distribusyon ay kontrolado ng `num_samples`, at isang pinagsama na coefficient ang ibinibigay para sa bawat natatanging sample. Para sa higit pang impormasyon tungkol sa kung paano kinakalkula ang mga coefficient, sumangguni sa [explanatory material](https://qiskit.github.io/qiskit-addon-cutting/explanation/index.html).

In [7]:
# Transpile the decomposed circuit to the same layout
transpiled_qpd_circuit = pm.run(subexperiments[100])

print(f"Original circuit depth after transpile: {transpiled_qc.depth()}")
print(
    f"QPD subexperiment depth after transpile: {transpiled_qpd_circuit.depth()}"
)
transpiled_qpd_circuit.draw(
    "mpl", scale=0.6, style="iqp", idle_wires=False, fold=-1
)

Original circuit depth after transpile: 103
QPD subexperiment depth after transpile: 46


<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/70e2f1b6-1.avif" alt="Output of the previous code cell" />

*Para sa paghahambing, nakikita natin na ang mga QPD subexperiment ay magiging mas mababa pagkatapos ng pagputol ng mga malayong gate*: Narito ang isang halimbawa ng isang arbitraryong piniling subexperiment na nabuo mula sa QPD circuit. Ang lalim nito ay nabawasan ng higit sa kalahati. Maraming ganitong probabilistic subexperiment ang dapat buuin at suriin upang muling itayo ang expectation value ng mas malalim na circuit.

In [8]:
print(f"Sampling overhead: {np.prod([basis.overhead for basis in bases])}")

Sampling overhead: 729.0


## Step 3: Execute using Qiskit primitives

Execute the target circuits ("subexperiments") with the Sampler Primitive.

*   Input: Target circuits
*   Output: Quasi-probability distributions

In [9]:
# Transpile the subexperiments to the backend's instruction set architecture (ISA)
isa_subexperiments = pm.run(subexperiments)

# Set up the Qiskit Runtime Sampler primitive.  For a fake backend, this will use a local simulator.
sampler = SamplerV2(backend)

# Submit the subexperiments
job = sampler.run(isa_subexperiments)

In [11]:
# Retrieve the results
results = job.result()

In [10]:
print(job.job_id())

czypg1r6rr3g008mgp6g


## Hakbang 3: Isagawa gamit ang Qiskit primitives
Isagawa ang mga target circuit ("subexperiments") gamit ang Sampler Primitive.

*   Input: Mga target circuit
*   Output: Mga quasi-probability distribution

In [12]:
reconstructed_expvals = reconstruct_expectation_values(
    results,
    coefficients,
    observables,
)
# Reconstruct final expectation value
final_expval = np.dot(reconstructed_expvals, [1] * len(observables))
print("Final reconstructed expectation value")
print(final_expval)

Final reconstructed expectation value
1.0751342773437473


In [13]:
ideal_expvals = [
    Statevector(circuit).expectation_value(SparsePauliOp(observable))
    for observable in observables
]
print("Ideal expectation value")
print(np.dot(ideal_expvals, [1] * len(observables)).real)

Ideal expectation value
1.2283177520039992


## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_2ftYFf9t72yFNIO)